In [1]:
using TensorFlow
using Distributions
using MLDataUtils
using SwiftObjectStores
using Iterators
using ColoringNames
using MappedArrays
using MLLabelUtils
using StaticArrays
using Base.Test
using ProgressMeter
using IJulia

Base.one{S<:AbstractString}(::Type{S}) = ""

In [2]:
serv=SwiftService()
train_raw = get_file(serv, "color", "monroe/train.csv") do fh
    readdlm(fh,'\t')
end

1523108×4 Array{Any,2}:
 "acid green"  0.344113  0.855319  0.921569
 "acid green"  0.281905  0.744681  0.921569
 "acid green"  0.347551  0.921397  0.898039
 "acid green"  0.311209  0.900398  0.984314
 "acid green"  0.312085  0.992095  0.992157
 "acid green"  0.22766   0.987395  0.933333
 "acid green"  0.288462  0.466368  0.87451 
 "acid green"  0.276936  0.804878  0.964706
 "acid green"  0.325359  0.95      0.862745
 "acid green"  0.385201  0.971774  0.972549
 "acid green"  0.212885  0.955823  0.976471
 "acid green"  0.346405  0.6       1.0     
 "acid green"  0.238397  0.42246   0.733333
 ⋮                                         
 "yuck"        0.1621    0.421965  0.678431
 "yuck"        0.153333  0.722543  0.678431
 "yuck"        0.201754  0.256757  0.580392
 "yuck"        0.264463  0.620513  0.764706
 "yuck"        0.144608  0.790698  0.67451 
 "yuck"        0.178125  0.97561   0.643137
 "yuck"        0.166667  0.350962  0.815686
 "yuck"        0.174603  0.640244  0.643137
 "yuck" 

In [3]:
tokenize = morpheme_tokenizer("../data/replacement_rules.csv")
demarcate(tokens) = ["<S>"; tokens; "</S>"]

@test demarcate(["a", "b", "c"]) == ["<S>", "a", "b", "c", "</S>"]

Test Passed
  Expression: demarcate(["a","b","c"]) == ["<S>","a","b","c","</S>"]
   Evaluated: String["<S>","a","b","c","</S>"] == String["<S>","a","b","c","</S>"]

In [4]:
keep_inds = shuffle(1:size(train_raw, 1))[1:5]

labels = convert(Vector{String}, train_raw[keep_inds,1]); #TODO use all data
hsv_data = convert(Matrix{Float64}, train_raw[keep_inds,2:end]); #TODO use all data


5×3 Array{Float64,2}:
 0.00793651  0.832599  0.890196
 0.220044    0.676991  0.886275
 0.488774    0.807531  0.937255
 0.411523    0.344681  0.921569
 0.176236    0.863636  0.94902 

In [5]:
tokenised_labels = demarcate.(tokenize.(labels))
all_tokens = reduce(union, tokenised_labels)
encoding = labelenc(all_tokens)
label_inds = map(x->label2ind.(x, Scalar(encoding)), tokenised_labels)

5-element Array{Array{Int64,1},1}:
 [1,2,3]  
 [1,4,5,3]
 [1,4,6,3]
 [1,6,3]  
 [1,7,3]  

In [6]:
function Base.rpad{T}(xs::Vector{T}, n::Integer, p::T=zero(T))
    sizehint!(xs, n)
    while length(xs)<n
        push!(xs, p)
    end
    xs
end

"Creates a matrix where each column is one of the vectors from `xss`"
function rpad_to_matrix{T}(xss::Vector{Vector{T}}, n_rows::T=maximum(length.(xss)), p::T=zero(T))
    n_cols = length(xss) 
    ret = fill(p, (n_rows, n_cols))
    for (cc, xs) in enumerate(xss)
        for (rr, x) in enumerate(xs)
            @inbounds ret[rr, cc] = x
        end
    end
    ret
end

@testset "padding" begin
    @test rpad([1,2,3],5) == [1,2,3,0,0]
    @test rpad([1,2,3],2) == [1,2,3]
    @test rpad([1,2,3],5, 8 ) == [1,2,3,8,8]
    
    @test rpad_to_matrix([[1,2], [3],Int[],[2]])==[
                                                    1 3 0 2
                                                    2 0 0 0
    ]
end;

Test Summary: | Pass  Total


In [7]:
padded_labels = rpad_to_matrix(label_inds)

4×5 Array{Int64,2}:
 1  1  1  1  1
 2  4  4  6  7
 3  5  6  3  3
 0  3  3  0  0

In [8]:
learning_rate = 0.05
n_input = 3 # HSV
@show n_steps, batch_size = size(padded_labels)
n_steps-=1 #can remove one demarcation
hidden_layer_size = 256
embedding_dim = 16
n_classes = nlabel(encoding)+1

(n_steps,batch_size) = size(padded_labels) = (4,5)


In [9]:
get_mask(V, dtype=Float32)=cast(cast(V, Bool),dtype)
apply_mask(V, mask) = V.*tile(expand_dims(mask, 1), [1, get_shape(V,2)])

@testset "masks" begin
    mask_sess = Session(Graph())
    a_val = (
    [2.  5  8  9  2
     3.  6  4  4  9
     4.  7   3  2  4
     1.  4  2  4  4]
    )
    A=constant(a_val)
    M = constant([1, 0, 2, 4])
    run(mask_sess, initialize_all_variables())
    masked = run(mask_sess, apply_mask(A, get_mask(M, Float64)))
    @test all(masked[2,:] .== 0)
    @test all(masked[1,:] .== a_val[1,:])
    @test all(masked[3:4,:] .== a_val[3:4,:])
end;

Test Summary: | Pass  

W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


Total
  masks       |    3      3


In [97]:
sess = Session(Graph())
X_hsv = placeholder(Float32, shape=[batch_size, n_input]; name="X_HSVs")
Term_obs_s = placeholder(Int32, shape=[n_steps+1, batch_size]; name="Term_obs_s")

EmbeddingTable = get_variable("TokenEmbeddings",  [n_classes, embedding_dim], Float32; initializer=Normal(0, .1))

#Mangle Terms into shape
Term_obs_s_out = slice(Term_obs_s, [1,0], [-1,-1]) #Don't want first input "<S>" #0based
Term_obs_s_ins = unpack(Term_obs_s+1)[1:end-1]#Don't want last input "</S>" (or padding character often but we will handle that seperately)
Term_obs_s_ins_r = reshape.(Term_obs_s_ins, Scalar(batch_size))
Tes = gather.(Scalar(EmbeddingTable), Term_obs_s_ins_r)
    
TT = reshape(Term_obs_s_out, [n_steps*batch_size]; name="Stack_Term_obs_s")
Term_obs_onehots = one_hot(TT, n_classes)


#Mangle colots into shape
X_h, X_s, X_v = unpack(X_hsv; axis=2)
X_hr = X_h.*2π
X_col = pack((sin(X_hr), cos(X_hr), X_s-0.5, X_v-0.5)) #Smooth hue by breaking into cos and sin, and zero mean everything else
X_col = reshape(X_col, [batch_size, 4]; name = "X_col") #Force Shape Inference
Xs = [concat(1, (T, X_col); name="Xs$ii") for (ii,T) in enumerate(Tes)]#Pair color input at each step with previous term #0base

Hs, states = nn.rnn(nn.rnn_cell.LSTMCell(hidden_layer_size), Xs; dtype=Float32)#, sequence_length=n_steps);
W1 = get_variable("weights1", [hidden_layer_size, n_classes], Float32;  initializer=Normal(0, .1))
B1 = get_variable("bias1", [n_classes], Float32;  initializer=Normal(0, .1))
Ls =  [H*W1+B1 for H in Hs]


LL = concat(0, Ls; name="Stack_Logits") #Stack #0base
LL = reshape(LL, [batch_size*n_steps, n_classes]) #Force Shape Inference
LL_masked = apply_mask(LL, get_mask(TT))

Term_preds_onehots = nn.softmax(LL_masked; name="Term_preds_onehots")
Term_preds_onehots_log = nn.log_softmax(LL_masked; name="Term_preds_onehots_log")
Term_preds_s = indmax(Term_preds_onehots, 2)


costs  = reduce_sum(Term_obs_onehots.*Term_preds_onehots_log, reduction_indices=[1])

#costs = nn.sparse_softmax_cross_entropy_with_logits(LL, TT+1)
cost = reduce_mean(-costs) #cross entropy

optimizer = train.minimize(train.AdamOptimizer(learning_rate), cost)

run(sess, initialize_all_variables())
run(sess, cost, Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))


3.1400099f0

In [100]:
global Term_preds_s_o
global TT_s_o
global cost_o
global optimizer_o

rounds = 50
#p = Progress(rounds)
@showprogress for i in 1:rounds
    Term_preds_s_o, TT_s_o, cost_o, optimizer_o = run(sess,
        [Term_preds_s, TT, cost, optimizer], 
        Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
    
end

@show cost_o
obs,pred, oo, pp = run(sess, [Term_obs_onehots, Term_preds_onehots, Term_obs_s_out, Term_preds_s], Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
    

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
cost_o = 0.0002957467f0


4-element Array{Array,1}:
 Float32[0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]                                                                           
 Float32[2.68795f-20 0.999708 … 0.000292125 3.32278f-20; 3.83931f-15 8.87451f-7 … 1.66134f-5 1.13468f-15; … ; 0.125 0.125 … 0.125 0.125; 0.125 0.125 … 0.125 0.125]
 Int32[2 4 … 6 7; 3 5 … 3 3; 0 3 … 0 0]                                                                                                                            
 [1,3,3,5,6,2,4,5,2,2,0,2,2,0,0]                                                                                                                                   

In [102]:
obs

15×8 Array{Float32,2}:
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [104]:
oo

3×5 Array{Int32,2}:
 2  4  4  6  7
 3  5  6  3  3
 0  3  3  0  0

In [112]:
reshape(mapslices(indmax,pred,2), (5,3))'

3×5 Array{Int64,2}:
 2  4  4  6  7
 3  5  6  3  3
 1  3  3  1  1

In [113]:
ind2label.(ans, Scalar(encoding))

3×5 Array{String,2}:
 "terracotta"  "light"  "light"  "teal"  "yellow"
 "</S>"        "green"  "teal"   "</S>"  "</S>"  
 "<S>"         "</S>"   "</S>"   "<S>"   "<S>"   

In [101]:
ind2label.(filter(x->x>0, Term_preds_s_o), Scalar(encoding))

12-element Array{String,1}:
 "<S>"       
 "</S>"      
 "</S>"      
 "green"     
 "teal"      
 "terracotta"
 "light"     
 "green"     
 "terracotta"
 "terracotta"
 "terracotta"
 "terracotta"

In [39]:
## MASK TEST######################
ll, ll2 = run(sess, [ LL_masked, 
    reshape(tile(expand_dims(get_mask(Term_obs_s_out),2),[1,1,8]).*concat(0, expand_dims.(Ls, Scalar(0))), [batch_size*n_steps, n_classes])
]
, Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
@test ll ≈ ll2

Test Passed
  Expression: ll ≈ ll2
   Evaluated: Float32[0.0960852 0.0510797 … 0.0918193 0.129112; 0.0922533 0.0635156 … 0.0912102 0.133232; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] isapprox Float32[0.0960852 0.0510797 … 0.0918193 0.129112; 0.0922533 0.0635156 … 0.0912102 0.133232; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [ ]:
methods(nn.sparse_softmax_cross_entropy_with_logits)